## Basic Testing

In [1]:
import numpy as np
dataset = np.load('./LondonAQ.npz')
dataset

In [3]:
ls = dataset.files
ls

['GEltham', 'GWesthorne', 'BSladeGreen', 'GWoolwich']

In [4]:
dataset[ls[0]].shape

(43848, 9)

In [5]:
dataset[ls[0]][0]

array([12.4,  0. ,  5.7, 54.4,  2.8,  1. ,  1. ,  6. ,  0. ])

## Convert dataset to input

In [6]:
vars = [0, 1, 2, 3, 4]
airq = {}
for d in ls:
        airq[d] = dataset[d]
        if vars is not None:
            airq[d] = airq[d][:, vars]

In [11]:
data = airq[ls[0]][:, 0].reshape(-1, 1)  ## We only use the first one feature from 9 features here.
data.shape

(43848, 1)

In [13]:
datasize = 35064
testsize = 8784
lag = 6
ahead = 1

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = scaler.fit_transform(data)

In [15]:
wind_train = data[:datasize, :]
wind_train.shape

(35064, 1)

### Executing below codes and trying to print train[0], train[1] and train[2]. Finding something from them!

In [26]:
ahead = 1 - 1
lvect = []
for i in range(lag):
    lvect.append(wind_train[i: -lag - ahead + i])
lvect.append(wind_train[lag + ahead:])
train = np.stack(lvect, axis=1)
train.shape

(35058, 7, 1)

In [27]:
train_x, train_y = train[:, :lag], train[:, -1:, 0]

In [29]:
wind_test = data[datasize:datasize + testsize, 0].reshape(-1, 1)
lvect = []
for i in range(lag):
    lvect.append(wind_test[i: -lag - ahead + i])
lvect.append(wind_test[lag + ahead:])
test = np.stack(lvect, axis=1)
test_x, test_y = test[:, :lag], test[:, -1:, 0]

### Now we get the data for our RNN: train_x, train_y, test_x, test_y

### Build RNN Model

In [34]:
def architecture(neurons, drop, nlayers, activation, activation_r, rnntype, impl=1):
    """
    RNN architecture

    :return:
    """
    RNN = LSTM if rnntype == 'LSTM' else GRU
    model = Sequential()
    if nlayers == 1:
        model.add(RNN(neurons, input_shape=(train_x.shape[1], train_x.shape[2]), implementation=impl,
                      recurrent_dropout=drop, activation=activation, recurrent_activation=activation_r))
    else:
        model.add(RNN(neurons, input_shape=(train_x.shape[1], train_x.shape[2]), implementation=impl,
                      recurrent_dropout=drop, activation=activation, recurrent_activation=activation_r,
                      return_sequences=True))
        for i in range(1, nlayers - 1):
            model.add(RNN(neurons, recurrent_dropout=drop, implementation=impl,
                          activation=activation, recurrent_activation=activation_r, return_sequences=True))
        model.add(RNN(neurons, recurrent_dropout=drop, activation=activation,
                      recurrent_activation=activation_r, implementation=impl))

    model.add(Dense(1))

    return model

### Load information from config.json

In [37]:
import json
def load_config_file(nfile, abspath=False):
    """
    Read the configuration from a json file

    :param abspath:
    :param nfile:
    :return:
    """
    ext = '.json' if 'json' not in nfile else ''
    pre = '' if abspath else './'
    fp = open(pre + nfile + ext, 'r')

    s = ''

    for l in fp:
        s += l

    return json.loads(s)

config = load_config_file('config')

In [40]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM, GRU

impl = 1
model = architecture(neurons=config['arch']['neurons'],
                         drop=config['arch']['drop'],
                         nlayers=config['arch']['nlayers'],
                         activation=config['arch']['activation'],
                         activation_r=config['arch']['activation_r'], rnntype=config['arch']['rnn'], impl=impl)

E:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [42]:
optimizer = config['training']['optimizer']

if optimizer == 'rmsprop':
    if 'lrate' in config['training']:
        optimizer = RMSprop(lr=config['training']['lrate'])
    else:
        optimizer = RMSprop(lr=0.001)

model.compile(loss='mean_squared_error', optimizer=optimizer)

In [44]:
cbacks = []
model.fit(train_x, train_y, batch_size=config['training']['batch'],
          epochs=config['training']['epochs'],
          validation_data=(test_x, test_y),
          verbose=False, callbacks=cbacks)

In [48]:
from sklearn.metrics import mean_squared_error, r2_score

score = model.evaluate(test_x, test_y, batch_size=config['training']['batch'], verbose=0)
print()
print('MSE test= ', score)
print('MSE test persistence =', mean_squared_error(test_y[ahead:], test_y[0:-ahead]))
test_yp = model.predict(test_x, batch_size=config['training']['batch'], verbose=0)
r2test = r2_score(test_y, test_yp)
r2pers = r2_score(test_y[ahead:, 0], test_y[0:-ahead, 0])
print('R2 test= ', r2test)
print('R2 test persistence =', r2pers)


MSE test=  0.19840409933814537


ValueError: Found input variables with inconsistent numbers of samples: [8778, 0]